## Traffic Stops
### Data Import and Cleaning

This notebook imports raw data on UCPD traffic stops scraped from the UCPD website. It takes several steps, including datetime extraction, categorical variable standardization, and merging on of additional variables for analysis (including latitude/longitudue and sunset times for time-of-day analysis). The output of the notebook is a processed dataset ready to be read in to the next notebook and further transformed for training a machine learning model.

The first step is to import relevant libraries for data munging, handling datetimes, performing API calls, and parsing JSON responses to API calls. The notebook also imports a utils file that contains my API key for the Google maps geocoding API, which I'm keeping secret for obvious (security) reasons. I also install and import Astral, which is a package containing data on sunset times.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
import requests
import json
from utils import API_KEY

In [2]:
%%capture
!pip install astral

In [3]:
from astral import sun
from astral import LocationInfo

Start by reading in and examining the raw data. Note that `datetime` is a raw string, `location` is an address fragment, and the `citation` field contains null values. 

In [4]:
traffic_stops = pd.read_csv('ucpd_traffic_stops.csv')
traffic_stops.head()

,datetime,location,race,gender,idot,reason,citation,disposition,search
0,1/9/2016 10:34 AM,1100 E 57th,Caucasian,Male,Traffic Sign/Signal,Disobeyed Stop Sign,NaN,Verbal Warning,No
1,1/12/2016` 4:16 PM,5800 S. Woodlawn,Caucasian,Female,Lane Violation,Driving on crosswalk,NaN,Verbal Warning,No
2,1/14/2016 10:28 AM,5700 S. University,Asian,Male,Lane Violation,Wrong way on one way street,NaN,Verbal Warning,No
3,1/14/2016 8:14 PM,6100 S. University,African American,Female,Lane Violation,Improper passing/overtaking,NaN,Verbal Warning,No
4,1/21/2016 8:49 PM,6000 S Drexel,African American,Male,Traffic Sign/Signal,Failed to stop at intersection with pedestrian...,Citation issued for disobeying stop sign,Citation Issued,No


#### Filter 
Visually inspecting the raw data reveals that the web scraper read in cells that contain the value `There are no traffic stops for [date]` in the `datetime`, so let's drop those observations. There's also an observation that contains an address (for Stony Island Ave), so we need to drop that, as well.

In [5]:
# Filter out observations corresponding to days with no traffic stops
# Drop observations that erroneously have location (and no date) in date field
traffic_stops = traffic_stops[~traffic_stops['datetime'].str.contains('There')]
traffic_stops = traffic_stops[~traffic_stops['datetime'].str.contains('Stony')]

#### Process datetime columns: fix typos, convert column types
Many cells have a semicolon `;` instead of a colon `:` in the `datetime` field, which will throw errors if we try to convert the field to type datetime. There's also an extraneous character of punctuation in some values of `datetime` that we need to clear out.

In [6]:
# Make some manual adjustments to typos, misspellings, etc.
traffic_stops['datetime'] = traffic_stops['datetime'].str.replace(pat=';', repl=':')
traffic_stops['datetime'] = traffic_stops['datetime'].str.replace(pat='`', repl='')

We can use Python's string `split()` method to split the datetime string into two pieces. The substring preceding the whitespace is the observation date and the substring after the whitespace is the observation time.

In [7]:
# Split datetime into separate date and time columns
traffic_stops[['date', 'time']] = traffic_stops['datetime'].str.split(" ", 1, expand=True)
traffic_stops.head()

,datetime,location,race,gender,idot,reason,citation,disposition,search,date,time
0,1/9/2016 10:34 AM,1100 E 57th,Caucasian,Male,Traffic Sign/Signal,Disobeyed Stop Sign,NaN,Verbal Warning,No,1/9/2016,10:34 AM
1,1/12/2016 4:16 PM,5800 S. Woodlawn,Caucasian,Female,Lane Violation,Driving on crosswalk,NaN,Verbal Warning,No,1/12/2016,4:16 PM
2,1/14/2016 10:28 AM,5700 S. University,Asian,Male,Lane Violation,Wrong way on one way street,NaN,Verbal Warning,No,1/14/2016,10:28 AM
3,1/14/2016 8:14 PM,6100 S. University,African American,Female,Lane Violation,Improper passing/overtaking,NaN,Verbal Warning,No,1/14/2016,8:14 PM
4,1/21/2016 8:49 PM,6000 S Drexel,African American,Male,Traffic Sign/Signal,Failed to stop at intersection with pedestrian...,Citation issued for disobeying stop sign,Citation Issued,No,1/21/2016,8:49 PM


We need to do some manual find-and-replace of bad `date` values before converting the `date` column to type `datetime`.

In [8]:
# More manual typo fixing in the date field
traffic_stops['date'] = traffic_stops['date'].str.replace(pat='9/152018', repl='9/15/2018')
traffic_stops['date'] = traffic_stops['date'].str.replace(pat='12/4/22018', repl='12/4/2018')
traffic_stops['date'] = traffic_stops['date'].str.replace(pat='10/27.2019', repl='10/27/2019')
traffic_stops['date'] = traffic_stops['date'].str.replace(pat='20019', repl='2019')

In [9]:
traffic_stops['date'] = pd.to_datetime(traffic_stops['date'], infer_datetime_format=True)
traffic_stops.head()

,datetime,location,race,gender,idot,reason,citation,disposition,search,date,time
0,1/9/2016 10:34 AM,1100 E 57th,Caucasian,Male,Traffic Sign/Signal,Disobeyed Stop Sign,NaN,Verbal Warning,No,2016-01-09,10:34 AM
1,1/12/2016 4:16 PM,5800 S. Woodlawn,Caucasian,Female,Lane Violation,Driving on crosswalk,NaN,Verbal Warning,No,2016-01-12,4:16 PM
2,1/14/2016 10:28 AM,5700 S. University,Asian,Male,Lane Violation,Wrong way on one way street,NaN,Verbal Warning,No,2016-01-14,10:28 AM
3,1/14/2016 8:14 PM,6100 S. University,African American,Female,Lane Violation,Improper passing/overtaking,NaN,Verbal Warning,No,2016-01-14,8:14 PM
4,1/21/2016 8:49 PM,6000 S Drexel,African American,Male,Traffic Sign/Signal,Failed to stop at intersection with pedestrian...,Citation issued for disobeying stop sign,Citation Issued,No,2016-01-21,8:49 PM


Repeat the process of fixing typos and converting the `time` column to the appropriate `time` type.

In [10]:
# Clean up time typos
traffic_stops['time'] = traffic_stops['time'].str.replace(pat='21:05 PM', repl='21:05')
traffic_stops['time'] = traffic_stops['time'].str.replace(pat='22:28 PM', repl='22:28')
traffic_stops['time'] = traffic_stops['time'].str.replace(pat='14:34 PM', repl='14:34')
traffic_stops['time'] = traffic_stops['time'].str.replace(pat='15:45 PM', repl='15:45')
traffic_stops['time'] = traffic_stops['time'].str.replace(pat='16:50 PM', repl='16:50')
traffic_stops['time'] = traffic_stops['time'].str.replace(pat='16:33 PM', repl='16:33')
traffic_stops['time'] = traffic_stops['time'].str.replace(pat='20:21 PM', repl='20:21')
traffic_stops['time'] = traffic_stops['time'].str.replace(pat='20:01 PM', repl='20:01')
traffic_stops['time'] = traffic_stops['time'].str.replace(pat='20:31 PM', repl='20:31')

In [11]:
traffic_stops['time'] = pd.to_datetime(traffic_stops['time'], infer_datetime_format=True).dt.time
traffic_stops.head()

/usr/local/lib/python3.6/dist-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname OM identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


,datetime,location,race,gender,idot,reason,citation,disposition,search,date,time
0,1/9/2016 10:34 AM,1100 E 57th,Caucasian,Male,Traffic Sign/Signal,Disobeyed Stop Sign,NaN,Verbal Warning,No,2016-01-09,10:34:00
1,1/12/2016 4:16 PM,5800 S. Woodlawn,Caucasian,Female,Lane Violation,Driving on crosswalk,NaN,Verbal Warning,No,2016-01-12,16:16:00
2,1/14/2016 10:28 AM,5700 S. University,Asian,Male,Lane Violation,Wrong way on one way street,NaN,Verbal Warning,No,2016-01-14,10:28:00
3,1/14/2016 8:14 PM,6100 S. University,African American,Female,Lane Violation,Improper passing/overtaking,NaN,Verbal Warning,No,2016-01-14,20:14:00
4,1/21/2016 8:49 PM,6000 S Drexel,African American,Male,Traffic Sign/Signal,Failed to stop at intersection with pedestrian...,Citation issued for disobeying stop sign,Citation Issued,No,2016-01-21,20:49:00


#### Process categorical columns: fix typos, collapse values into cohesive bins

Next we'll take a look at the categorical columns in the dataset and see whether we need to manually recode values. For example, in the counts table that follows, we see that there are multiple values for `race` that should really fall under a single "African American" category.

In [12]:
traffic_stops['race'].value_counts()

African American                          3216
Caucasian                                  712
Hispanic                                   215
Asian                                      205
African  American                           19
ASIAN                                       18
CAUCASIAN                                   17
American Indian/Alaskan Native               9
Native Hawaiian/Other Pacific Islander       4
American Indian                              2
African                                      1
American\nIndian/Alaskan Native              1
Name: race, dtype: int64

In [13]:
# Make corrections for race categories
traffic_stops.loc[traffic_stops['race'].isin(['African  American', 'African']), 'race'] = 'African American'
traffic_stops.loc[traffic_stops['race']=='CAUCASIAN', 'race'] = 'Caucasian'
traffic_stops.loc[traffic_stops['race']=='ASIAN', 'race'] = 'Asian'
traffic_stops.loc[traffic_stops['race'].isin(['American Indian/Alaskan Native',
                                              'American Indian',
                                              'American\nIndian/Alaskan Native']), 'race'] = 'American Indian/Alaska Native'
traffic_stops['race'].value_counts()

African American                          3236
Caucasian                                  729
Asian                                      223
Hispanic                                   215
American Indian/Alaska Native               12
Native Hawaiian/Other Pacific Islander       4
Name: race, dtype: int64

We can repeat the categorical variable recoding process for `gender` and `idot` (the category of incident according to the Illinois Department of Transportation). The `reason` field has 575 distinct values with more detailed incident descriptions, but in the interest of time, we can focus our analysis on the incident category as communicated by `idot`, so there's no need to standardize `reason`.

In [14]:
# Make corrections for gender categories
traffic_stops['gender'].value_counts()

Male      2819
Female    1594
female       4
male         1
Name: gender, dtype: int64

In [15]:
traffic_stops.loc[traffic_stops['gender']=='male', 'gender'] = 'Male'
traffic_stops.loc[traffic_stops['gender']=='female', 'gender'] = 'Female'
traffic_stops['gender'].value_counts()

Male      2820
Female    1598
Name: gender, dtype: int64

In [16]:
# Make corrections for idot
traffic_stops['idot'].value_counts()

Traffic Sign/Signal                       1617
Traffic sign/signal                       1026
Lane Violation                            1001
Equipment                                  242
Speed                                      161
Moving Violation - Other                   122
moving Violation - Other                    57
Moving Violation-Other                      43
Moving Violation � Other                    39
License Plate/Registration                  28
License plate/registration                  20
Seatbelt                                    18
Moving Violation  � Other                    9
moving Violation-Other                       7
License Plate Registration                   6
Moving Violation  Other                     4
Follow too close                             3
Seat belt                                    3
Moving Violation   Other                    2
Follow Too Close                             2
License Plate/registration                   1
moving Violat

In [17]:
traffic_stops.loc[traffic_stops['idot'].isin(['Traffic sign/signal',
                                              'Disobeyed redlight',
                                              'Failure to Yield  Pedestrian Crosswalk']), 'idot'] = 'Traffic Sign/Signal'
traffic_stops.loc[traffic_stops['idot'] == 'lane Violation', 'idot'] = 'Lane Violation'
traffic_stops.loc[traffic_stops['idot'].isin(['moving Violation - Other',
                                              'Moving Violation-Other',
                                              'Moving Violation � Other',
                                              'Moving Violation  � Other',
                                              'moving Violation-Other',
                                              'Moving Violation  Other',
                                              'Moving Violation   Other',
                                              'moving Violation-Otherv',
                                              'Moving  Violation  � Other']), 'idot'] = 'Moving Violation - Other'
traffic_stops.loc[traffic_stops['idot'].isin(['License plate/registration',
                                              'License Plate Registration',
                                              'license Plate Registration',
                                              'license Plate/Registration',
                                              'License Plate/registration']), 'idot'] = 'License Plate/Registration'
traffic_stops.loc[traffic_stops['idot'].isin(['Seat belt']), 'idot'] = 'Seatbelt'
traffic_stops.loc[traffic_stops['idot'].isin(['Follow Too Close',
                                              'Follow to close']), 'idot'] = 'Follow too close'
traffic_stops['idot'].value_counts()     

Traffic Sign/Signal           2645
Lane Violation                1002
Moving Violation - Other       285
Equipment                      242
Speed                          161
License Plate/Registration      57
Seatbelt                        21
Follow too close                 6
Name: idot, dtype: int64

In [18]:
# There are 575 distinct values for the 'reason' field. These are more detailed 
# descriptions of the specific incident that are already aggregated into categories
# in the 'idot' field, so we can ignore these for brevity's sake.
len(traffic_stops['reason'].unique())

575

The next field we need to clean is the `citation` field. Technically, this is a text column, but the information we want to extract from it is actually numeric (the number of citations issued). This number is reported in a non-standard and wordy way, so we have to develop some simple rules for pulling an integer value out of a string column.

In [19]:
# We need to extract the number of citations issued from the 'citation' field,
# which will be tricky to do because it's a messy text column
citation_values = traffic_stops['citation'].value_counts().rename_axis('citation').reset_index(name='counts')
pd.options.display.max_rows = 70
citation_values

,citation,counts
0,Two citations issued,112
1,One Citation Issued,111
2,One citation issued,87
3,Two Citations issued,63
4,Two Citations Issued,38
5,0,32
6,Three citations issued,18
7,Two Citation Issued,16
8,One citation issued for disobeying traffic con...,9
9,Four citations issued,8


Visual inspection of the distinct values of `citation` reveals that the word "Citation" always appears when some citation is issued, and for any number of citations issued greater than 1, some form of the number issued appears (either as an integer, e.g. '4', or as a string, e.g. 'four'). We can start by assigning 0 to all observations in a new `num_citations` column and then iteratively updating that value based on the string value found in the `citation` field. We can then do a visual double-check to make sure the new column accurately translates the information from the old column.

In [20]:
# Create a new column that extracts number of citations using a heuristic,
# which we can then double-check for accuracy
citation_values.loc[:, 'num_citations'] = 0
citation_values.loc[citation_values['citation'].str.contains('One'), 'num_citations'] = 1
citation_values.loc[citation_values['citation'].str.contains('Citation'), 'num_citations'] = 1
citation_values.loc[citation_values['citation'].str.contains('citation'), 'num_citations'] = 1
citation_values.loc[citation_values['citation'].str.contains('1'), 'num_citations'] = 1
citation_values.loc[citation_values['citation'].str.contains('Two'), 'num_citations'] = 2
citation_values.loc[citation_values['citation'].str.contains('two'), 'num_citations'] = 2
citation_values.loc[citation_values['citation'].str.contains('2'), 'num_citations'] = 2
citation_values.loc[citation_values['citation'].str.contains('Three'), 'num_citations'] = 3
citation_values.loc[citation_values['citation'].str.contains('three'), 'num_citations'] = 3
citation_values.loc[citation_values['citation'].str.contains('3'), 'num_citations'] = 3
citation_values.loc[citation_values['citation'].str.contains('Four'), 'num_citations'] = 4
citation_values.loc[citation_values['citation'].str.contains('four'), 'num_citations'] = 4
citation_values.loc[citation_values['citation'].str.contains('4'), 'num_citations'] = 4
citation_values.loc[citation_values['citation'].str.contains('Five'), 'num_citations'] = 5
citation_values.loc[citation_values['citation'].str.contains('five'), 'num_citations'] = 5
citation_values.loc[citation_values['citation'].str.contains('5'), 'num_citations'] = 5
citation_values.loc[citation_values['citation'].str.contains('Six'), 'num_citations'] = 6
citation_values.loc[citation_values['citation'].str.contains('six'), 'num_citations'] = 6
citation_values.loc[citation_values['citation'].str.contains('6'), 'num_citations'] = 6
citation_values = citation_values.drop(labels='counts', axis=1)
citation_values

,citation,num_citations
0,Two citations issued,2
1,One Citation Issued,1
2,One citation issued,1
3,Two Citations issued,2
4,Two Citations Issued,2
5,0,0
6,Three citations issued,3
7,Two Citation Issued,2
8,One citation issued for disobeying traffic con...,1
9,Four citations issued,4


We created a new data frame with the numeric `num_citations` field, so now we have to merge it back on to the main dataframe using the character `citations` field as the key for the left join.

In [21]:
# Merge 'num_citations' back onto main dataframe, fill null num_citations value with 0
traffic_stops_merged = traffic_stops.merge(citation_values, how='left', on='citation')
traffic_stops_merged['num_citations'] = traffic_stops_merged['num_citations'].fillna(0)
traffic_stops = traffic_stops_merged
traffic_stops.head(20)

,datetime,location,race,gender,idot,reason,citation,disposition,search,date,time,num_citations
0,1/9/2016 10:34 AM,1100 E 57th,Caucasian,Male,Traffic Sign/Signal,Disobeyed Stop Sign,NaN,Verbal Warning,No,2016-01-09,10:34:00,0.0
1,1/12/2016 4:16 PM,5800 S. Woodlawn,Caucasian,Female,Lane Violation,Driving on crosswalk,NaN,Verbal Warning,No,2016-01-12,16:16:00,0.0
2,1/14/2016 10:28 AM,5700 S. University,Asian,Male,Lane Violation,Wrong way on one way street,NaN,Verbal Warning,No,2016-01-14,10:28:00,0.0
3,1/14/2016 8:14 PM,6100 S. University,African American,Female,Lane Violation,Improper passing/overtaking,NaN,Verbal Warning,No,2016-01-14,20:14:00,0.0
4,1/21/2016 8:49 PM,6000 S Drexel,African American,Male,Traffic Sign/Signal,Failed to stop at intersection with pedestrian...,Citation issued for disobeying stop sign,Citation Issued,No,2016-01-21,20:49:00,1.0
5,1/22/2016 7:32 PM,5626 S Blackstone,African American,Male,Traffic Sign/Signal,Running stop sign and speeding,NaN,Verbal Warning,No,2016-01-22,19:32:00,0.0
6,1/26/2016 10:42 PM,5500 Dorchester,African American,Male,Traffic Sign/Signal,Ran stop sign,NaN,Verbal Warning,No,2016-01-26,22:42:00,0.0
7,1/30/2016 2:41 AM,5100 Drexel Ave,African American,Male,Traffic Sign/Signal,Ran red light,NaN,Verbal Warning,No,2016-01-30,02:41:00,0.0
8,2/5/2016 6:16 PM,5950 S Woodlawn,African American,Male,Traffic Sign/Signal,Ran red light,NaN,Verbal Warning,No,2016-02-05,18:16:00,0.0
9,2/5/2016 9:25 PM,6200 S Stony Island,African American,Male,Traffic Sign/Signal,Ran stop sign,NaN,Verbal Warning,No,2016-02-05,21:25:00,0.0


Next we perform the same category-collapsing for values of the `disposition` field (for example, "Verbal Warning" and "verbal Warning" are treated by the dataframe as two distinct values, but they really should be the same).

In [22]:
traffic_stops['disposition'].value_counts()

Verbal Warning                                 3867
Citation Issued                                 347
Citation issued                                 153
Citations Issued                                 18
verbal Warning                                   17
Citations issued                                  7
Citation  Issued                                  2
Arrest                                            1
Moving Violation-Other                            1
Arrested by CPD                                   1
Citation issued; arrested on warrant              1
Arrested on warrant                               1
Citation Issued, Arrested on Active Warrant       1
Arrested                                          1
Name: disposition, dtype: int64

In [23]:
# Fix disposition categories
traffic_stops.loc[traffic_stops['disposition'].isin(['Verbal Warning', 'verbal Warning']), 'disposition_cat'] = 'Verbal Warning'
traffic_stops.loc[traffic_stops['disposition'].isin(['Citation Issued', 'Citation issued', 'Citations Issued', 'Citations issued', 'Citation  Issued']), 'disposition_cat'] = 'Citation Issued'
traffic_stops.loc[traffic_stops['disposition'].isin(['Arrested on warrant', 'Arrested by CPD', 'Arrested', 'Arrest']), 'disposition_cat'] = 'Arrested'
traffic_stops.loc[traffic_stops['disposition'].isin(['Citation Issued, Arrested on Active Warrant', 'Citation issued; arrested on warrant']), 'disposition_cat'] = 'Citation Issued; Arrested'

# Add binary outcome cols
traffic_stops.loc[:, 'verbal_warning'] = 0
traffic_stops.loc[traffic_stops['disposition']=='Verbal Warning', 'verbal_warning'] = 1
traffic_stops.loc[:, 'citation_issued'] = 0
traffic_stops.loc[traffic_stops['disposition'].isin(['Citation Issued', 'Citation Issued; Arrested']), 'citation_issued'] = 1
traffic_stops.loc[:, 'arrested'] = 0
traffic_stops.loc[traffic_stops['disposition'].isin(['Arrested', 'Citation Issued; Arrested']), 'arrested'] = 1

traffic_stops.head(20)

,datetime,location,race,gender,idot,reason,citation,disposition,search,date,time,num_citations,disposition_cat,verbal_warning,citation_issued,arrested
0,1/9/2016 10:34 AM,1100 E 57th,Caucasian,Male,Traffic Sign/Signal,Disobeyed Stop Sign,NaN,Verbal Warning,No,2016-01-09,10:34:00,0.0,Verbal Warning,1,0,0
1,1/12/2016 4:16 PM,5800 S. Woodlawn,Caucasian,Female,Lane Violation,Driving on crosswalk,NaN,Verbal Warning,No,2016-01-12,16:16:00,0.0,Verbal Warning,1,0,0
2,1/14/2016 10:28 AM,5700 S. University,Asian,Male,Lane Violation,Wrong way on one way street,NaN,Verbal Warning,No,2016-01-14,10:28:00,0.0,Verbal Warning,1,0,0
3,1/14/2016 8:14 PM,6100 S. University,African American,Female,Lane Violation,Improper passing/overtaking,NaN,Verbal Warning,No,2016-01-14,20:14:00,0.0,Verbal Warning,1,0,0
4,1/21/2016 8:49 PM,6000 S Drexel,African American,Male,Traffic Sign/Signal,Failed to stop at intersection with pedestrian...,Citation issued for disobeying stop sign,Citation Issued,No,2016-01-21,20:49:00,1.0,Citation Issued,0,1,0
5,1/22/2016 7:32 PM,5626 S Blackstone,African American,Male,Traffic Sign/Signal,Running stop sign and speeding,NaN,Verbal Warning,No,2016-01-22,19:32:00,0.0,Verbal Warning,1,0,0
6,1/26/2016 10:42 PM,5500 Dorchester,African American,Male,Traffic Sign/Signal,Ran stop sign,NaN,Verbal Warning,No,2016-01-26,22:42:00,0.0,Verbal Warning,1,0,0
7,1/30/2016 2:41 AM,5100 Drexel Ave,African American,Male,Traffic Sign/Signal,Ran red light,NaN,Verbal Warning,No,2016-01-30,02:41:00,0.0,Verbal Warning,1,0,0
8,2/5/2016 6:16 PM,5950 S Woodlawn,African American,Male,Traffic Sign/Signal,Ran red light,NaN,Verbal Warning,No,2016-02-05,18:16:00,0.0,Verbal Warning,1,0,0
9,2/5/2016 9:25 PM,6200 S Stony Island,African American,Male,Traffic Sign/Signal,Ran stop sign,NaN,Verbal Warning,No,2016-02-05,21:25:00,0.0,Verbal Warning,1,0,0


In [24]:
traffic_stops['search'].value_counts()

No     4013
NO      301
Yes      78
YES      13
no        2
Name: search, dtype: int64

Repeat the category collapsing process for the `search` column. This is more straightforward, as the only fixes we need to make are accounting for values that are in all caps as opposed to title case.

In [25]:
traffic_stops.loc[traffic_stops['search'].isin(['No', 'NO', 'no']), 'search'] = 0
traffic_stops.loc[traffic_stops['search'].isin(['Yes', 'YES']), 'search'] = 1
traffic_stops['search'].value_counts()

0    4316
1      91
Name: search, dtype: int64

In [26]:
traffic_stops = traffic_stops[['date', 'time', 'location', 'gender', 'race', 'idot', 'verbal_warning', 'citation_issued', 'num_citations', 'search', 'arrested']]
traffic_stops.head()

,date,time,location,gender,race,idot,verbal_warning,citation_issued,num_citations,search,arrested
0,2016-01-09,10:34:00,1100 E 57th,Male,Caucasian,Traffic Sign/Signal,1,0,0.0,0,0
1,2016-01-12,16:16:00,5800 S. Woodlawn,Female,Caucasian,Lane Violation,1,0,0.0,0,0
2,2016-01-14,10:28:00,5700 S. University,Male,Asian,Lane Violation,1,0,0.0,0,0
3,2016-01-14,20:14:00,6100 S. University,Female,African American,Lane Violation,1,0,0.0,0,0
4,2016-01-21,20:49:00,6000 S Drexel,Male,African American,Traffic Sign/Signal,0,1,1.0,0,0


#### Geocode addresses using the Google Maps API
For geocoding purposes, we need to expand the address fragments provided by UCPD into a full address with city and state information. Luckily, all of these addresses are within Chicago city limits, so we don't need a fancy solution - all we have to do is use string concatenation to add "Chicago, IL" to the end of each address value.

In [27]:
traffic_stops.loc[:, 'full_address'] = traffic_stops['location'].map(str) + ", Chicago, IL"
traffic_stops.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,date,time,location,gender,race,idot,verbal_warning,citation_issued,num_citations,search,arrested,full_address
0,2016-01-09,10:34:00,1100 E 57th,Male,Caucasian,Traffic Sign/Signal,1,0,0.0,0,0,"1100 E 57th, Chicago, IL"
1,2016-01-12,16:16:00,5800 S. Woodlawn,Female,Caucasian,Lane Violation,1,0,0.0,0,0,"5800 S. Woodlawn, Chicago, IL"
2,2016-01-14,10:28:00,5700 S. University,Male,Asian,Lane Violation,1,0,0.0,0,0,"5700 S. University, Chicago, IL"
3,2016-01-14,20:14:00,6100 S. University,Female,African American,Lane Violation,1,0,0.0,0,0,"6100 S. University, Chicago, IL"
4,2016-01-21,20:49:00,6000 S Drexel,Male,African American,Traffic Sign/Signal,0,1,1.0,0,0,"6000 S Drexel, Chicago, IL"


Next, we'll use the Google Maps API to geocode these addresses and turn them into latitude/longitude coordinates. These coordinates will be useful for plotting traffic stops on a map and calculating sunset and dusk times for daylight analyses. We start by defining the base URL for the Google maps API and getting a list of unique addresses to query the API with - since the dataframe contains multiple stops in the same location, we can save time and effort if we only geocode the unique addresses and then merge them back on to the main dataframe.

In [28]:
# Pull down lat/lon and formatted address using google maps API
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?address='
addresses_to_geocode = traffic_stops['full_address'].unique().tolist()

Iterate through the unique address values and query the Google Maps API. Pull out the fully `formatted_address` field as well as `lat` and `lng`. Once we have all the coordinates we can save the objejct as a dataframe.

In [29]:
observations = []
for address in addresses_to_geocode[930:]:
  obs = {'full_address': address}
  request_url = base_url + '+'.join(address.split(' ')) + "&key=" + API_KEY
  r = requests.get(request_url)
  address_dict = json.loads(r.text)
  try:
    obs['formatted_address'] = address_dict['results'][0]['formatted_address']
  except:
    obs['formatted_address'] = ''
  try:
    obs['lat'] = address_dict['results'][0]['geometry']['location']['lat']
  except:
    obs['lat'] = np.nan
  try:
    obs['lng'] = address_dict['results'][0]['geometry']['location']['lng']
  except:
    obs['lng'] = np.nan
  observations.append(obs)

In [30]:
geocoded_df = pd.DataFrame(observations)
geocoded_df.head()

,full_address,formatted_address,lat,lng
0,"5118 S Lake Park, Chicago, IL",,NaN,NaN
1,"5200 S. Harper, Chicago, IL","5200 S Harper Ave, Chicago, IL 60615, USA",41.800830,-87.589508
2,"5100 S. Greenwood, Chicago, IL","5100 S Greenwood Ave, Chicago, IL 60615, USA",41.802306,-87.599812
3,"6200 S. Park Shore East, Chicago, IL","6200 S Park Shore E Ct, Chicago, IL 60637, USA",41.782444,-87.588167
4,"5700 S. Cornell, Chicago, IL","5700 S Cornell Ave, Chicago, IL 60637, USA",41.793008,-87.585454


Next we merge `latitude`, `longitude` and the `formatted_address` field back onto the main dataframe.

In [31]:
traffic_stops_merged = traffic_stops.merge(geocoded_df, how='left', on='full_address')
traffic_stops = traffic_stops_merged
traffic_stops.head()

,date,time,location,gender,race,idot,verbal_warning,citation_issued,num_citations,search,arrested,full_address,formatted_address,lat,lng
0,2016-01-09,10:34:00,1100 E 57th,Male,Caucasian,Traffic Sign/Signal,1,0,0.0,0,0,"1100 E 57th, Chicago, IL",NaN,NaN,NaN
1,2016-01-12,16:16:00,5800 S. Woodlawn,Female,Caucasian,Lane Violation,1,0,0.0,0,0,"5800 S. Woodlawn, Chicago, IL",NaN,NaN,NaN
2,2016-01-14,10:28:00,5700 S. University,Male,Asian,Lane Violation,1,0,0.0,0,0,"5700 S. University, Chicago, IL",NaN,NaN,NaN
3,2016-01-14,20:14:00,6100 S. University,Female,African American,Lane Violation,1,0,0.0,0,0,"6100 S. University, Chicago, IL",NaN,NaN,NaN
4,2016-01-21,20:49:00,6000 S Drexel,Male,African American,Traffic Sign/Signal,0,1,1.0,0,0,"6000 S Drexel, Chicago, IL",NaN,NaN,NaN


In [32]:
# Check to see how many formatted addresses are null
len(traffic_stops[traffic_stops['formatted_address']==''])

1

#### Adding sunset/dusk times with the Astral package
Now we need to use the GPS coordinates we gathered in the previous step along with `date` information to find sunset and dusk times for the "veil of darkness" analysis. The Astral package allows users to define a city, time zone and GPS coordinates and then look up all relevant astronomicaly times for a given date. We'll do this for every latitude-longitude-date triplet in the dataset and create a dataset of sunset times to merge back onto the main dataframe.

In [33]:
sunset_observations = []
for row in traffic_stops.iterrows():
  lat = row[1]['lat']
  lng = row[1]['lng']
  date = row[1]['date']
  city = LocationInfo("Chicago", "United States", "America/Chicago", lat, lng)
  s = sun.sun(city.observer, date=date, tzinfo=city.timezone)
  s['dawn'] = s['dawn'].time()
  s['dusk'] = s['dusk'].time()
  s['noon'] = s['noon'].time()
  s['sunrise'] = s['sunrise'].time()
  s['sunset'] = s['sunset'].time()
  s['lat'] = lat
  s['lng'] = lng
  s['date'] = date
  sunset_observations.append(s)

ValueError: ignored

In [ ]:
sunset_df = pd.DataFrame(sunset_observations)
sunset_df.head()

In [ ]:
traffic_stops_merged = traffic_stops.merge(sunset_df, how='left', on=['lat','lng','date'])
traffic_stops_merged.head()

In [ ]:
traffic_stops = traffic_stops_merged[['date', 'time', 'full_address', 'formatted_address', 'lat', 'lng', 'gender', 
                                      'race', 'idot', 'verbal_warning', 'citation_issued',
                                      'num_citations', 'search', 'arrested', 'dawn', 'sunrise',
                                      'noon', 'sunset', 'dusk']]
traffic_stops.head()

The necessary cleaning steps are done, so we can export the dataframe to a CSV and prepare to use it analysis.

In [ ]:
traffic_stops.to_csv('ucpd_traffic_stops_processed.csv', index=False)